# SQL Query using Peewee ORM

In this notebook, we will learn how to Query using Peewee ORM

For sample database, we will use [Chinook SQLite Sample Database](https://www.sqlitetutorial.net/sqlite-sample-database/)

The following database diagram illustrates the chinook database tables and their relationships.

![](https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg)

---

## Convert SQLite DB to Data Models

As mentioned in Previous notebook, in order to use Peewee ORM, all tables need to be converted to Data Models using python class.

But since we already had the DB with all the table and relationships, we can use [Peewee Extension called **pwiz**](https://docs.peewee-orm.com/en/latest/peewee/playhouse.html#pwiz) To convert the DB to Data Models.

In [9]:
# This code will convert sqlite db to pewee models in file peewee_chinook_models.py
!python -m pwiz -e sqlite -i database/chinook.db > peewee_chinook_models.py

## Manually Recreate Table from DB

We can also manually recreate the peewee model table from the DB

In [7]:
from peewee import *

database = SqliteDatabase('database/chinook.db')

class BaseModel(Model):
    class Meta:
        database = database
        
class Artists(BaseModel):
    artist_id = AutoField(column_name='ArtistId')
    name = CharField(120, column_name='Name', null=True)  # NVARCHAR(120)

    class Meta:
        table_name = 'artists'
        
class Albums(BaseModel):
    album_id = AutoField(column_name='AlbumId')
    artist = ForeignKeyField(column_name='ArtistId', field='artist_id', model=Artists)
    title = CharField(160, column_name='Title')  # NVARCHAR(160)

    class Meta:
        table_name = 'albums'
                
class Genres(BaseModel):
    genre_id = AutoField(column_name='GenreId')
    name = CharField(120, column_name='Name', null=True)  # NVARCHAR(120)

    class Meta:
        table_name = 'genres'     

class MediaTypes(BaseModel):
    media_type_id = AutoField(column_name='MediaTypeId')
    name = CharField(120, column_name='Name', null=True)  # NVARCHAR(120)

    class Meta:
        table_name = 'media_types'
        
class Tracks(BaseModel):
    album = ForeignKeyField(column_name='AlbumId', field='album_id', model=Albums, null=True)
    bytes = IntegerField(column_name='Bytes', null=True)
    composer = CharField(220, column_name='Composer', null=True)  # NVARCHAR(220)
    genre = ForeignKeyField(column_name='GenreId', field='genre_id', model=Genres, null=True)
    media_type = ForeignKeyField(column_name='MediaTypeId', field='media_type_id', model=MediaTypes)
    milliseconds = IntegerField(column_name='Milliseconds')
    name = CharField(200, column_name='Name')  # NVARCHAR(200)
    track_id = AutoField(column_name='TrackId')
    unit_price = DecimalField(column_name='UnitPrice')

    class Meta:
        table_name = 'tracks'                           

## Querying using ORM Peewee

After successfully extract the Data Models from the DB, we can now use the Data Models to query the DB.

In [1]:
import sqlite3
import pandas as pd

## Query Comparison between SQL and ORM

1. Select all from Artist table

In [12]:
orm_query = Artists.select().dicts()

pd.DataFrame(orm_query)

,artist_id,name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains
...,...,...
270,271,"Mela Tenenbaum, Pro Musica Prague & Richard Kapp"
271,272,Emerson String Quartet
272,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
273,274,Nash Ensemble


In [13]:
print(orm_query.sql())

('SELECT "t1"."ArtistId", "t1"."Name" FROM "artists" AS "t1"', [])


We can convert ORM query to SQL query using `query.sql()` method

<br>


2. Select Songs that are longer than 3 minutes

In [14]:
# Select Songs that are longer than 3 minutes
orm_query = Tracks.select().where(Tracks.milliseconds > 180000).dicts()
print(orm_query.sql())
pd.DataFrame(orm_query)

('SELECT "t1"."TrackId", "t1"."AlbumId", "t1"."Bytes", "t1"."Composer", "t1"."GenreId", "t1"."MediaTypeId", "t1"."Milliseconds", "t1"."Name", "t1"."UnitPrice" FROM "tracks" AS "t1" WHERE ("t1"."Milliseconds" > ?)', [180000])


,track_id,album,bytes,composer,genre,media_type,milliseconds,name,unit_price
0,1,1,11170334,"Angus Young, Malcolm Young, Brian Johnson",1,1,343719,For Those About To Rock (We Salute You),0.99
1,2,2,5510424,None,1,2,342562,Balls to the Wall,0.99
2,3,3,3990994,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",1,2,230619,Fast As a Shark,0.99
3,4,3,4331779,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",1,2,252051,Restless and Wild,0.99
4,5,3,6290521,Deaffy & R.A. Smith-Diesel,1,2,375418,Princess of the Dawn,0.99
...,...,...,...,...,...,...,...,...,...
3018,3497,341,4307907,None,24,2,261849,"Erlkonig, D.328",0.99
3019,3498,342,16454937,Pietro Antonio Locatelli,24,4,493573,"Concerto for Violin, Strings and Continuo in G...",0.99
3020,3499,343,4718950,None,24,2,286741,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,0.99
3021,3502,346,3665114,Wolfgang Amadeus Mozart,24,2,221331,"Quintet for Horn, Violin, 2 Violas, and Cello ...",0.99


3. How many songs that are longer than 3 minutes and shorter than 5 minutes

In [18]:
orm_query = Tracks.select().where((Tracks.milliseconds > 180000) & (Tracks.milliseconds < 300000)).count()

print(f'There are {orm_query} songs that longer than 3 minutes and shorter than 5 minutes')

There are 1954 songs that longer than 3 minutes and shorter than 5 minutes


4. Select All songs in Album "Let There Be Rock" by AC/DC	

In [25]:
# Select All songs in Album "Let There Be Rock"
orm_query = (
    Tracks
        .select(
            Albums.title, 
            Tracks.composer, 
            Tracks.name, 
            Tracks.unit_price)
        .join(Albums)
        .where(
            Albums.title == 'Let There Be Rock'
        )
        .dicts()
)
print(orm_query.sql())
pd.DataFrame(orm_query)

('SELECT "t1"."Title", "t2"."Composer", "t2"."Name", "t2"."UnitPrice" FROM "tracks" AS "t2" INNER JOIN "albums" AS "t1" ON ("t2"."AlbumId" = "t1"."AlbumId") WHERE ("t1"."Title" = ?)', ['Let There Be Rock'])


,title,composer,name,unit_price
0,Let There Be Rock,AC/DC,Go Down,0.99
1,Let There Be Rock,AC/DC,Dog Eat Dog,0.99
2,Let There Be Rock,AC/DC,Let There Be Rock,0.99
3,Let There Be Rock,AC/DC,Bad Boy Boogie,0.99
4,Let There Be Rock,AC/DC,Problem Child,0.99
5,Let There Be Rock,AC/DC,Overdose,0.99
6,Let There Be Rock,AC/DC,Hell Ain't A Bad Place To Be,0.99
7,Let There Be Rock,AC/DC,Whole Lotta Rosie,0.99


5. Select Top 5 Artist with most songs

In [30]:
# Select Top 5 Artist with most songs
orm_query = (
    Artists
        .select(
            Artists.name, 
            fn.COUNT(Tracks.track_id).alias('song_count')
        )
        .join(Albums)
        .join(Tracks)
        .group_by(Artists.name)
        .order_by(
            fn.COUNT(Tracks.track_id).desc()
        )
        .limit(5)
        .dicts()
)
print(orm_query.sql())
pd.DataFrame(orm_query)

('SELECT "t1"."Name", COUNT("t2"."TrackId") AS "song_count" FROM "artists" AS "t1" INNER JOIN "albums" AS "t3" ON ("t3"."ArtistId" = "t1"."ArtistId") INNER JOIN "tracks" AS "t2" ON ("t2"."AlbumId" = "t3"."AlbumId") GROUP BY "t1"."Name" ORDER BY COUNT("t2"."TrackId") DESC LIMIT ?', [5])


,name,song_count
0,Iron Maiden,213
1,U2,135
2,Led Zeppelin,114
3,Metallica,112
4,Lost,92
